In [1]:
import pandas as pd
from flags import FLAGS, unparsed
import numpy as np
from db.conn_db import db,cursor,engine,truncate_table,data_from_mysql,dev_id_train
import operator  
from functools import reduce

In [2]:
deviceid_train=dev_id_train()

In [3]:
deviceid_packages= pd.read_csv(FLAGS.file_path +'deviceid_packages.csv',)

In [4]:
def app_list(text):
    app_list=text.split('|')
#        print (app_list)
    return app_list
deviceid_packages['app_list']=deviceid_packages['add_id_list'].apply(lambda line:app_list(line)).tolist()

In [5]:
deviceid_train=pd.merge(deviceid_train,deviceid_packages,on=['device_id'],how='left') 

In [6]:
deviceid_train.shape

(50000, 6)

In [9]:
def difference_list(app_list):
    import copy
    retlist=[]
    for i in range(0,len(app_list)):
        
        alist=app_list[i]
        tlist=copy.deepcopy(app_list)
        tlist.remove(alist)
        blist=reduce(operator.add, tlist)
        clist=list(set(alist).difference(set(blist)))
        retlist.append(clist)
    return retlist

In [7]:
apps_list=[]
all_app=reduce(operator.add, deviceid_packages.app_list.tolist())
train_app=reduce(operator.add, deviceid_train.app_list.tolist())
no_train_app=list(set(all_app).difference(set(train_app)))
for i in range(0,23):
    type_list1=deviceid_train.ix[deviceid_train['n_class'].values==i,'app_list'].tolist()
#        print(i)
    if len(type_list1)<1:
        apps_list.append([])
    else:
        type_list1=reduce(operator.add, type_list1)
        apps_list.append(type_list1)

/home/zhijiehuang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [19]:
def intersection_list(app_list):
    import copy
    retlist=[]
    for i in range(0,len(app_list)):
        
        alist=app_list[i]
        tlist=copy.deepcopy(app_list)
        tlist.remove(alist)
        blist=reduce(operator.add, tlist)
        clist=list(set(alist).intersection(set(blist)))
        retlist.append(clist)
    return retlist

In [20]:
inter_list=intersection_list(apps_list)

In [22]:
inter_list

[['c972020c351e4b96c12af23aad6f56f5',
  'f3e682493e1898d8b10c6a19ea7595fd',
  '7e63332f0624b1c438eb73f6f0a55e28',
  '7ee5697469a9d1ef745d9193ef95d341',
  'e795eb9c2daac1eb6b7ad5a90530bfe0',
  '07e967d75aab2f6a52c558695a572a7c',
  'c67faa0589255fad8559af83b52ff86e',
  '94bd984e8679c17aa69eb7f9a87b3fc1',
  '963322d6a4dd4146d0695c71882e96ec',
  'e3350a88a808781942abb4c02e316076',
  'f141bb0a3447193aa7359c2c89fd7549',
  '5228986cb754e8b754cc234ffa217e86',
  'eb978b9ab973cb1fcc5837cd91657275',
  '8ec8f62d4694d88c24baacb1f15a9673',
  '4a2f764d29d62ec22d8eb1787293ccc5',
  '5df3c72209d881801c395b023384de55',
  'fa5cf5e15605344647f46979a04ea434',
  '2c94bf3242575e85879b036c109b39ea',
  'e9d21564509ba24dcbb55c8a07ac4a06',
  '8c8544b6c129ad4a431be753143ed1c3',
  '45bdd7dc63b10c247082e11ab3d5a371',
  'cc28c21bc755a5faf11ff91665ddcd92',
  'f268e158b0f394767f1c67296f0b7eee',
  '6e32d9a42976b263347a17c6c2c91c18',
  '3c4f17d40cb1d559ae821c90da6d0ad1',
  'e6a45c8c85eedd5c780f6a309898b1dc',
  '09519ea0e

In [41]:
np.unique(deviceid_train.n_class.values==0)

array([False,  True])

In [49]:
app_dict={}
def c(a,b):
    #print(str(a),b)
    ert=(str(a) in b)
    return ert
i=0
for i,x in enumerate(apps_list):
    for li in x:
        _x=[]
        for _ in range(deviceid_train.shape[0]):
            _x.append(li)
#        print(_x)
        _x=pd.DataFrame({'a':_x},dtype='category')
        
        a=list(map(c,_x['a'],deviceid_train['app_list']))

        filte1=np.logical_and(deviceid_train.n_class.values==i,True)
        print(deviceid_train.ix[filte1,'app_list'].shape[0])
        filte2=np.logical_and(a,True)
        print(deviceid_train.ix[filte2,'app_list'].shape[0])
        app_dict[li]=deviceid_train.ix[filte1,'app_list'].shape[0]/deviceid_train.ix[filte2,'app_list'].shape[0]
        break
    break


109
8103


In [ ]:
deviceid_packages['app3_w']=deviceid_packages['app_list'].apply(lambda x:app_dict_get(x,app_dict))